In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if ollama_api_key:
    print(f"OLLAMA API Key exists and begins {ollama_api_key[:2]}")
else:
    print("OLLAMA API Key not set (and this is optional)")

# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key=ollama_api_key, base_url=ollama_url)



In [ ]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "gemini-2.5-pro", "gpt-oss:20b-cloud", ]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "gemini-2.5-pro": gemini,  "gpt-oss:20b-cloud": ollama}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [ ]:
system_prompt_doc = """You are an expert Python developer and code reviewer.
Your job is to read the user's provided function, and return:
1. A concise, PEP-257-compliant docstring summarizing what the function does, clarifying types, parameters, return values, and side effects.
2. Helpful inline comments that improve both readability and maintainability, without restating what the code obviously does.

Only output the function, not explanations or additional text. 
Do not modify variable names or refactor the function logic.
Your response should improve the code's clarity and documentation, making it easier for others to understand and maintain.
Don't be extremely verbose.
Your answer should be at a {level} level of expertise.
"""

system_prompt_tests = """You are a seasoned Python developer and testing expert.
Your task is to read the user's provided function, and generate:
1. A concise set of meaningful unit tests that thoroughly validate the function's correctness, including typical, edge, and error cases.
2. The tests should be written for pytest (or unittest if pytest is not appropriate), use clear, descriptive names, and avoid unnecessary complexity.
3. If dependencies or mocking are needed, include minimal necessary setup code (but avoid over-mocking).

Only output the relevant test code, not explanations or extra text.
Do not change the original function; focus solely on comprehensive, maintainable test coverage that other developers can easily understand and extend.
"""


In [ ]:
def generate_documentation(code, model, level):
    response = clients[model].chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_doc.format(level=level)},
            {"role": "user", "content": code}
        ],
        stream=True
    )
    output = ""
    for chunk in response:
        output += chunk.choices[0].delta.content or ""
        yield output.replace("```python", "").replace("```", "")



In [ ]:
def generate_tests(code, model ):
    response = clients[model].chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_tests},
            {"role": "user", "content": code}
        ],
        stream=True
    )
    output = ""
    for chunk in response:
        output += chunk.choices[0].delta.content or ""
        yield output.replace("```python", "").replace("```", "")



In [ ]:
vscode_dark = gr.themes.Monochrome(
    primary_hue="blue",
    secondary_hue="slate",
    neutral_hue="slate",
).set(
    body_background_fill="#1e1e1e",
    body_background_fill_dark="#1e1e1e",
    block_background_fill="#252526",
    block_background_fill_dark="#252526",
    block_border_color="#3e3e42",
    block_border_color_dark="#3e3e42",
    border_color_primary="#3e3e42",
    block_label_background_fill="#252526",
    block_label_background_fill_dark="#252526",
    block_label_text_color="#cccccc",
    block_label_text_color_dark="#cccccc",
    block_title_text_color="#cccccc",
    block_title_text_color_dark="#cccccc",
    body_text_color="#d4d4d4",
    body_text_color_dark="#d4d4d4",
    button_primary_background_fill="#0e639c",
    button_primary_background_fill_dark="#0e639c",
    button_primary_background_fill_hover="#1177bb",
    button_primary_background_fill_hover_dark="#1177bb",
    button_primary_text_color="#ffffff",
    button_primary_text_color_dark="#ffffff",
    input_background_fill="#3c3c3c",
    input_background_fill_dark="#3c3c3c",
    color_accent="#007acc",
    color_accent_soft="#094771",
)


In [ ]:
import gradio as gr

with gr.Blocks(theme=vscode_dark, css="""
    .gradio-container {font-family: 'Consolas', 'Monaco', monospace;}
    h1 {color: #d4d4d4 !important;}
""") as ui:
    gr.Markdown("# 🧑‍💻 Python Code Reviewer & Test Generator", elem_id="app-title")
    with gr.Tab("Docstring & Comments") as tab1:
        gr.Markdown("# Function Docstring & Comment Helper\nPaste your function below and get helpful docstrings and inline comments!")

        with gr.Row():
            code_input_1 = gr.Code(label="Paste your Python function here", lines=10, language="python")
            code_output = gr.Code(label="Function with improved docstring and comments", lines=10, language="python")
        
        with gr.Row(equal_height=True):
            level_radio = gr.Radio(choices=["Junior", "Mid", "Senior"], value="Mid", label="Reviewer level", interactive=True)
            model_dropdown = gr.Dropdown(choices=models, value=models[-1], label="Select model")
            submit_doc_btn = gr.Button("Generate docstring & comments", scale=0.5)

        submit_doc_btn.click(
            generate_documentation, 
            inputs=[code_input_1, model_dropdown, level_radio], 
            outputs=code_output
        )

    with gr.Tab("Unit Tests") as tab2:
        gr.Markdown("# Unit Test Generator\nPaste your function below and get auto-generated unit tests!")

        with gr.Row():
            code_input_2 = gr.Code(label="Paste your Python function here", lines=10, language="python")
            code_output_2 = gr.Code(label="Generated tests", lines=10, language="python")
        
        with gr.Row(equal_height=True):
            model_dropdown_2 = gr.Dropdown(choices=models, value=models[-1], label="Select model")
            submit_test_btn = gr.Button("Generate unit tests", scale=0.5)

        submit_test_btn.click(
            generate_tests, 
            inputs=[code_input_2, model_dropdown_2], 
            outputs=code_output_2
        )
    
    tab2.select(lambda x: x, inputs=code_input_1, outputs=code_input_2)
    tab1.select(lambda x: x, inputs=code_input_2, outputs=code_input_1)

ui.launch(share=False, inbrowser=True)